In [1]:
%load_ext magic_duckdb

In [2]:
%%dql -t df
INSTALL httpfs;

,Success


In [3]:
%%dql -t df
LOAD  httpfs;

,Success


# Data Check

In [4]:
%%dql
-- First, create the schema if it doesn't already exist
CREATE SCHEMA IF NOT EXISTS modealto;


,Count


In [5]:
%%dql
CREATE OR REPLACE TABLE modealto.order_status AS 
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/MarkPhamm/Alto-Pharmacy-Case-Study/main/data/order_status.csv');

,Count
0,7156


In [6]:
%%dql
CREATE OR REPLACE TABLE modealto.checkout_attempts AS 
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/MarkPhamm/Alto-Pharmacy-Case-Study/main/data/checkout_attempts.csv');

,Count
0,11992


In [8]:
%%dql
SELECT *
FROM modealto.order_status 
LIMIT 5

,order_id,customer_id,delivery_status,delivered_date
0,10000.0,8389.0,delivered,2023-11-04
1,10001.0,6811.0,delivered,2023-11-02
2,10002.0,9441.0,delivered,2023-11-03
3,10003.0,1125.0,delivered,2023-11-03
4,10004.0,7838.0,delivered,2023-11-02


In [9]:
%%dql
SELECT *
FROM modealto.checkout_attempts
LIMIT 5

,customer_id,checkout_started_at,checkout_completed_at,order_id,customer_added_non_med_to_order,order_total_dollars
0,1177.0,2023-11-01 06:45:18,NaT,NaN,NaN,NaN
1,6119.0,2023-11-01 06:47:35,NaT,NaN,NaN,NaN
2,8389.0,2023-11-01 06:50:25,2023-11-01 07:05:16,10000.0,False,11.75
3,6811.0,2023-11-01 06:55:26,2023-11-01 06:59:06,10001.0,False,19.83
4,9441.0,2023-11-01 06:59:34,2023-11-01 07:06:17,10002.0,False,21.59


# 1. Non-Medication Items Add-to-Cart Rate


## 1.1 Before checkouts

In [10]:
%%dql -t df
SELECT 
  (COUNT(case when customer_added_non_med_to_order = TRUE THEN 1 else NULL END)+0.0)/COUNT(*)*100 as non_med_added_rate
FROM 
  modealto.checkout_attempts

,non_med_added_rate
0,3.260507


In [13]:
%%dql -t df
SELECT 
  EXTRACT(YEAR FROM checkout_started_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_started_at) as check_out_month,
  (COUNT(case when customer_added_non_med_to_order = TRUE THEN 1 else NULL END)+0.0)/COUNT(*)*100 as non_med_added_rate
FROM 
  modealto.checkout_attempts
GROUP BY 
  EXTRACT(YEAR FROM checkout_started_at), 
  EXTRACT(MONTH FROM checkout_started_at)
ORDER BY 
  EXTRACT(YEAR FROM checkout_started_at), 
  EXTRACT(MONTH FROM checkout_started_at)

,check_out_year,check_out_month,non_med_added_rate
0,2023,11,0.000000
1,2023,12,4.597701
2,2024,1,6.433824


## 1.2 After checkouts

In [12]:
%%dql -t df
SELECT 
  (COUNT(case when customer_added_non_med_to_order = TRUE THEN 1 else NULL END)+0.0)/COUNT(*)*100 as non_med_added_rate
FROM 
  modealto.checkout_attempts
WHERE 
  checkout_completed_at IS NOT NULL;

,non_med_added_rate
0,5.463946


In [14]:
%%dql -t df
SELECT 
  EXTRACT(YEAR FROM checkout_started_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_started_at) as check_out_month,
  (COUNT(case when customer_added_non_med_to_order = TRUE THEN 1 else NULL END)+0.0)/COUNT(*)*100 as non_med_added_rate
FROM 
  modealto.checkout_attempts
WHERE 
  checkout_completed_at IS NOT NULL
GROUP BY 
  EXTRACT(YEAR FROM checkout_started_at), 
  EXTRACT(MONTH FROM checkout_started_at)
ORDER BY 
  EXTRACT(YEAR FROM checkout_started_at), 
  EXTRACT(MONTH FROM checkout_started_at)

,check_out_year,check_out_month,non_med_added_rate
0,2023,11,0.000000
1,2023,12,7.703281
2,2024,1,10.574018


# 2. Checkout rate

In [17]:
%%dql -t df
SELECT 
    ROUND((COUNT(checkout_completed_at) + 0.0) / COUNT(*) * 100, 2) AS check_out_rate
FROM 
    modealto.checkout_attempts;

,check_out_rate
0,59.67


In [18]:
%%dql -t df
SELECT 
  EXTRACT(YEAR FROM checkout_started_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_started_at) as check_out_month,
  ROUND((COUNT(checkout_completed_at)+0.0)/COUNT(*)*100,2) as check_out_rate
FROM 
  modealto.checkout_attempts
GROUP BY 
  EXTRACT(YEAR FROM checkout_started_at),
  EXTRACT(MONTH FROM checkout_started_at) 
ORDER BY 
  EXTRACT(YEAR FROM checkout_started_at),
  EXTRACT(MONTH FROM checkout_started_at) 

,check_out_year,check_out_month,check_out_rate
0,2023,11,58.96
1,2023,12,59.68
2,2024,1,60.85


# 3. Delivery Rate

In [19]:
%%dql -t df
SELECT 
    ROUND((COUNT(delivered_date)+0.0)/COUNT(*)*100,2) AS delivery_rate
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING (order_id)

,delivery_rate
0,93.14


In [20]:
%%dql -t df
-- Delivery Rate Break Down By Month
SELECT 
  EXTRACT(YEAR FROM checkout_completed_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_completed_at) as check_out_month,
  ROUND((COUNT(CASE WHEN delivery_status = 'delivered' THEN 1 ELSE NULL END)+0.0)/COUNT(*)*100,2) AS delivery_rate
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING (order_id)
GROUP BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 
ORDER BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 

,check_out_year,check_out_month,delivery_rate
0,2023,11,93.36
1,2023,12,94.12
2,2024,1,91.12


In [21]:
%%dql -t df
SELECT  
  AVG(CASE WHEN customer_added_non_med_to_order = TRUE THEN order_total_dollars ELSE NULL END) as non_med_added_avg_order_value,
  AVG(CASE WHEN customer_added_non_med_to_order = FALSE THEN order_total_dollars ELSE NULL END) as non_med_not_added_avg_order_value
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING(order_id)
WHERE
  delivery_status = 'delivered';

,non_med_added_avg_order_value,non_med_not_added_avg_order_value
0,30.163955,19.960825


# 4. Revenue and Number of Orders

In [22]:
%%dql -t df
SELECT  
  EXTRACT(YEAR FROM checkout_completed_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_completed_at) as check_out_month,
  SUM(order_total_dollars) as Revenue, 
  COUNT(*) as Number_of_Order
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING(order_id)
WHERE
  delivery_status = 'delivered'
GROUP BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 
ORDER BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 

,check_out_year,check_out_month,Revenue,Number_of_Order
0,2023,11,50145.62,2518
1,2023,12,55172.17,2639
2,2024,1,31662.67,1508
